In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import *
from fn import *



class OMSQ1:
    def __init__(self,user='',password='',host='',dbname='',table = False):
        self.U = user
        self.P = password
        self.H = host
        self.D = dbname
        self.tableinfo = {}
        self.conn = ''
        self.cn = ''
        self.dbtype = ''
        self.tbl = table
        self.df = pd.DataFrame([])
        self.qdf = pd.DataFrame([])
        self.qrydfmain = pd.DataFrame([])
        self.qrydf = pd.DataFrame([])
    def oracle_default(self):
        self.conn = cx_Oracle.connect('SOC_READ','soc_read', 'ossam-cluster-scan.robi.com.bd:1721/RBPB.robi.com.bd')
        print(self.conn)
    def oracle_custom(self):
        self.conn = cx_Oracle.connect(self.U,self.P,'ossam-cluster-scan.robi.com.bd:1721/RBPB.robi.com.bd')
        print(self.conn)
    def mssql(self):
        cstr2 = "mssql+pyodbc://" + self.U + ':' + self.P + '@' + self.H + "/" + self.D + "?driver=SQL Server"
        self.conn = create_engine(cstr2)
        self.dbtype = 'MSSQL'
        print('conn succ mssql')
    def pyodbc_mssql(self):
        cstr = "Driver={SQL Server};SERVER=" + self.H + ";DATABASE=" + self.D + ";UID=" + self.U + ";PWD=" + self.P
        self.cn = pyodbc.connect(cstr)
        self.dbtype = 'MSSQL'
        print('conn succ mssql')
    def mysql(self):
        constr = 'mysql+mysqlconnector://' + self.U + ':' + self.P + '@' + self.H + '/' + self.D
        self.conn = create_engine(constr, echo=False)
        self.cn = self.conn.raw_connection()
        self.dbtype = 'MYSQL'
        print('conn succ mysql')
    def table_info(self, tbl=False):
        if tbl == False:
            tbl = self.tbl
        if self.dbtype == 'MSSQL':
            self.tableinfo = mssql_table_colinfo(tbl,self.conn)
        elif self.dbtype == 'MYSQL':
            self.tableinfo = mysql_table_colinfo(self.D, tbl, self.conn)
        else:
            print('No DB initiated yet')
    def ex(self, qry):
        self.qdf = pd.read_sql(qry, con = self.conn)
        return qdf
    def get_conn_oracle(self):
        return self.conn
    def get_conn_other(self):
        return self.cn
    def set_df(self,ndf):
        self.df = ndf
    def check_table(self, tbl):
        return is_table_exist(self.conn,self.table)
    def get_tableinfo(self):
        return self.tableinfo
    def get_qrydf(self):
        return self.qrydf
    def create_tbl(self,df,tbl):
        if self.dbtype == 'MSSQL':
            self.pyodbc_mssql()
            msq.CreateTable_MSSQL(df, tbl, self.cn)
        elif self.dbtype == 'MYSQL':
            myq.CreateTable_MYSQL(connection = self.cn, tablename = tbl, df = df)
    def export_prep(self, ndf, tbl, bycol=[], sep=','):
        self.tbl = tbl
        self.table_info(self.tbl)
        df = ndf.fillna('$')
        df = mod_datetime(df)
        self.qrydfmain = df
        df = rmv_space_in_cols(self.qrydfmain)
        ls1 = insert_into(df)
        lss1 = ['INSERT INTO ' + tbl + i for i in ls1]
        if bycol is None or bycol==[]:
            df1 = pd.DataFrame(ls1, columns=['INS'])
            df1['INS'] = df1.apply(lambda x : 'INSERT INTO ' + tbl + x['INS'], axis = 1)
            self.qrydf = df1
        else:
            ls2 = update_into(df,bycols=bycol)
            dfn = df[bycol]
            ls3 = update_into(dfn,bycols=[],seperator=sep,operator='=')
            lss2 = ['UPDATE ' + tbl + ' SET ' + i + ' WHERE ' + j for i, j in zip(ls2, ls3)]
            self.qrydf = pd.DataFrame(zip(lss2,lss1),columns=['UPD','INS'])
    def export_execute(self, ifexist='fail'):
        try:
            self.qrydfmain.to_sql(self.tbl, con=self.conn, if_exists=ifexist, chunksize=10000)
            print('try success')
        except:
            if self.dbtype == 'MSSQL':
                self.pyodbc_mssql()
            cr = self.cn.cursor()
            for i in range(len(self.qrydf)):
                for j in self.qrydf:
                    q = self.qrydf.loc[i,j]
                    x = q.find(" WHERE ")
                    if x != -1:
                        tb = q.find(self.tbl)
                        qq = 'select 1 from ' + self.tbl + ' ' + q[x:len(q)]
                        ddf = pd.read_sql(qq,con=self.conn)
                        if ddf.shape[0]>0:
                            try:
                                cr.execute(q)
                                break
                            except:
                                print('failed: ', q)
                    else:
                        try:
                            cr.execute(q)
                            break
                        except:
                            print('failed: ', q)
            else:
                self.cn.commit()
                cr.close()
                print('over')
    def cursor_execute(self, qry):
        cr = self.cn.cursor()
        rs = cr.execute(qry)
        return rs
    def engin_execute(self, qry):
        df = pd.read_sql(qry, con= self.conn)
        return df











